<a href="https://colab.research.google.com/github/justorfc/DataAspirant_codes/blob/master/02_Notebook_Maestro_Entrevistas_Analisis_Total.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Maestro: Análisis temático y de sentimiento en entrevistas

Este notebook permite:
- Procesar múltiples entrevistas (.txt)
- Calcular frecuencias y exportarlas a .csv
- Construir una matriz documento-término (TF-IDF)
- Realizar análisis de sentimiento
- Aplicar clustering temático
- Visualizar resultados

## 1. Instalar e importar librerías necesarias

In [ ]:
!pip install -q nltk scikit-learn textblob
!python -m textblob.download_corpora

import nltk
import os
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

nltk.download('stopwords')
stopwords_esp = set(stopwords.words('spanish'))

## 2. Subir múltiples archivos `.txt`

In [ ]:
from google.colab import files
uploaded = files.upload()
txt_files = [f for f in uploaded if f.endswith('.txt')]

## 3. Procesar cada archivo: frecuencias, sentimiento y guardar CSV

In [ ]:
datos = []
corpus = []

for archivo in txt_files:
    with open(archivo, 'r', encoding='utf-8') as f:
        texto = f.read()
        corpus.append(texto)

    # Tokenización simple
    palabras = re.findall(r'\b\w+\b', texto.lower())
    tokens = [p for p in palabras if p not in stopwords_esp]
    frecuencias = Counter(tokens)

    # Exportar CSV por archivo
    df_frec = pd.DataFrame(frecuencias.items(), columns=['palabra', 'frecuencia'])
    df_frec['archivo'] = archivo
    df_frec.to_csv(archivo.replace('.txt', '_frecuencias.csv'), index=False)

    # Análisis de sentimiento con TextBlob
    sentimiento = TextBlob(texto).sentiment.polarity

    # Guardar datos básicos
    datos.append({
        'archivo': archivo,
        'texto': texto,
        'tokens': ' '.join(tokens),
        'sentimiento': sentimiento
    })

df_datos = pd.DataFrame(datos)
df_datos.to_csv("resumen_entrevistas.csv", index=False)

## 4. Construir matriz documento-término (TF-IDF)

In [ ]:
vectorizador = TfidfVectorizer()
tfidf = vectorizador.fit_transform(df_datos['tokens'])
matriz_tfidf = pd.DataFrame(tfidf.toarray(), columns=vectorizador.get_feature_names_out())
matriz_tfidf['archivo'] = df_datos['archivo']
matriz_tfidf.head()

## 5. Análisis de Clustering con KMeans

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0)
clusters = kmeans.fit_predict(tfidf)
df_datos['cluster'] = clusters

## 6. Visualización con PCA

In [ ]:
pca = PCA(n_components=2)
coords = pca.fit_transform(tfidf.toarray())

plt.figure(figsize=(10, 6))
sns.scatterplot(x=coords[:,0], y=coords[:,1], hue=df_datos['cluster'], palette='Set2')
for i, archivo in enumerate(df_datos['archivo']):
    plt.text(coords[i,0]+0.01, coords[i,1]+0.01, archivo[:10], fontsize=9)
plt.title("Agrupamiento de entrevistas (PCA + KMeans)")
plt.grid(True)
plt.show()

## 7. Visualización del sentimiento por entrevista

In [ ]:
plt.figure(figsize=(10,4))
sns.barplot(data=df_datos, x='archivo', y='sentimiento', palette='coolwarm')
plt.xticks(rotation=90)
plt.title("Análisis de sentimiento por entrevista")
plt.grid(True)
plt.show()

## 8. Exportar datos finales

In [ ]:
df_datos.to_csv("resultados_finales_sentimiento_clusters.csv", index=False)
print("Exportado como resultados_finales_sentimiento_clusters.csv")

## 9. Conclusión

Este notebook maestro permite un análisis integrado de múltiples entrevistas: análisis textual, sentimiento, agrupamiento y visualizaciones. Puede extenderse a mapas temáticos, dashboards o análisis temporal si se requiere.